## 1. Tensorflow
### 1.2 tf.data

In [1]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras import preprocessing

##### Create data

In [2]:
samples = ['너 오늘 이뻐 보인다',
           '나는 오늘 기분이 더러워',
           '끝내주는데, 좋은 일이 있나봐',
           '나 좋은 일이 생겼어',
           '아 오늘 진짜 짜증나',
           '환상적인데, 정말 좋은거 같아']

label = [[1], [0], [1], [1], [0], [1]] # It doesn't have meaning.

In [3]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
word_index = tokenizer.word_index

print("Mapping numerical data: \n", sequences)
print("\nIndex for each word: \n", word_index)
print("\nLabel: \n", label)

Mapping numerical data: 
 [[4, 1, 5, 6], [7, 1, 8, 9], [10, 2, 3, 11], [12, 2, 3, 13], [14, 1, 15, 16], [17, 18, 19, 20]]

Index for each word: 
 {'오늘': 1, '좋은': 2, '일이': 3, '너': 4, '이뻐': 5, '보인다': 6, '나는': 7, '기분이': 8, '더러워': 9, '끝내주는데': 10, '있나봐': 11, '나': 12, '생겼어': 13, '아': 14, '진짜': 15, '짜증나': 16, '환상적인데': 17, '정말': 18, '좋은거': 19, '같아': 20}

Label: 
 [[1], [0], [1], [1], [0], [1]]


##### With tf.data

In [4]:
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

- tf.data.Dataset.from_tensor_slices: It makes set of data matching sequences and label.

- make_one_shot_iterator: It allows us to use data one by one. 

- get_next: With get_next with iterator, it produces one data.

And all we defined above is tensor, hence, we need to implement with tf.Session

In [5]:
with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

(array([4, 1, 5, 6]), array([1]))
(array([7, 1, 8, 9]), array([0]))
(array([10,  2,  3, 11]), array([1]))
(array([12,  2,  3, 13]), array([1]))
(array([14,  1, 15, 16]), array([0]))
(array([17, 18, 19, 20]), array([1]))


Also, tf.data allows us to shuffle, batch or iteration way more easily.

##### Batch

In [6]:
BATCH_SIZE = 2

dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.batch(BATCH_SIZE)
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
     while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

(array([[4, 1, 5, 6],
       [7, 1, 8, 9]]), array([[1],
       [0]]))
(array([[10,  2,  3, 11],
       [12,  2,  3, 13]]), array([[1],
       [1]]))
(array([[14,  1, 15, 16],
       [17, 18, 19, 20]]), array([[0],
       [1]]))


##### Shuffle

In [7]:
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.shuffle(len(sequences)) 
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

(array([4, 1, 5, 6]), array([1]))
(array([14,  1, 15, 16]), array([0]))
(array([10,  2,  3, 11]), array([1]))
(array([17, 18, 19, 20]), array([1]))
(array([7, 1, 8, 9]), array([0]))
(array([12,  2,  3, 13]), array([1]))


##### Epoch

In [8]:
EPOCH = 3
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.repeat(EPOCH)
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

(array([4, 1, 5, 6]), array([1]))
(array([7, 1, 8, 9]), array([0]))
(array([10,  2,  3, 11]), array([1]))
(array([12,  2,  3, 13]), array([1]))
(array([14,  1, 15, 16]), array([0]))
(array([17, 18, 19, 20]), array([1]))
(array([4, 1, 5, 6]), array([1]))
(array([7, 1, 8, 9]), array([0]))
(array([10,  2,  3, 11]), array([1]))
(array([12,  2,  3, 13]), array([1]))
(array([14,  1, 15, 16]), array([0]))
(array([17, 18, 19, 20]), array([1]))
(array([4, 1, 5, 6]), array([1]))
(array([7, 1, 8, 9]), array([0]))
(array([10,  2,  3, 11]), array([1]))
(array([12,  2,  3, 13]), array([1]))
(array([14,  1, 15, 16]), array([0]))
(array([17, 18, 19, 20]), array([1]))


##### Mapping

In [9]:
def mapping_fn(X, Y=None):
    input = {'x': X}
    label = Y
    return input, label

dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.map(mapping_fn)
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

({'x': array([4, 1, 5, 6])}, array([1]))
({'x': array([7, 1, 8, 9])}, array([0]))
({'x': array([10,  2,  3, 11])}, array([1]))
({'x': array([12,  2,  3, 13])}, array([1]))
({'x': array([14,  1, 15, 16])}, array([0]))
({'x': array([17, 18, 19, 20])}, array([1]))


In [10]:
BATCH_SIZE = 2
EPOCH = 3

def mapping_fn(X, Y=None):
    input = {'x': X}
    label = Y
    return input, label

dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.map(mapping_fn)
dataset = dataset.shuffle(len(sequences))
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.repeat(EPOCH)
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

({'x': array([[10,  2,  3, 11],
       [ 7,  1,  8,  9]])}, array([[1],
       [0]]))
({'x': array([[17, 18, 19, 20],
       [14,  1, 15, 16]])}, array([[1],
       [0]]))
({'x': array([[ 4,  1,  5,  6],
       [12,  2,  3, 13]])}, array([[1],
       [1]]))
({'x': array([[12,  2,  3, 13],
       [17, 18, 19, 20]])}, array([[1],
       [1]]))
({'x': array([[14,  1, 15, 16],
       [ 4,  1,  5,  6]])}, array([[0],
       [1]]))
({'x': array([[ 7,  1,  8,  9],
       [10,  2,  3, 11]])}, array([[0],
       [1]]))
({'x': array([[10,  2,  3, 11],
       [17, 18, 19, 20]])}, array([[1],
       [1]]))
({'x': array([[ 4,  1,  5,  6],
       [14,  1, 15, 16]])}, array([[1],
       [0]]))
({'x': array([[ 7,  1,  8,  9],
       [12,  2,  3, 13]])}, array([[0],
       [1]]))
